<a href="https://colab.research.google.com/github/Georgia-MAX-holic/theory/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/%EC%95%99%EC%83%81%EB%B8%94%26%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 앙상블 학습 

- 학상블 학습 : 여러개의 분류기(Classifier)를 생성하고 그 예측을 결합함으로써 보다 정확한 최종 예측을 도출하는 기법 

- 다양한 분류기의 예측 결과를 결합함으로써 단일 분류기보다 신뢰성이 높은 예측값을 얻는 것 





---


### 앙상블의 유형 

- 앙상블의 유형은 일반적으로 보팅(Voting), 배깅(Bagging) , 부스팅(Boosting)으로 구분할 수 있으며 이외에 스태킹(Stacking)등의 기법이 있음 

- 대표적인 배깅은 랜덤 포레스트(Random Forest)알고리즘이 있으며, 부스팅은 에이다 부스팅, 그래디언트 부스팅 , XGBoost , LightGBM 등이 있음, 정형 데이터의 분류나 회귀에서는 GBM 부스팅 계열의 앙상블이 전반적으로 높은 예측 성능을 나타냄  

- 넓은 의미로는 서로 다른 모델을 결합한 것들을 앙상블로 지칭 하기도 함 



---

### 앙상블의 특징 

- 단일 모델의 약점을 다수의 모델들을 결합하여 보완 

- 뛰어난 성능을 가진 모델들로만 구성하는 것보다 성능이 떨어지더라도 서로 다른 유형의 모델을 섞는 것이 오히려 전체 성능에 도움이 될 수 있음 

- 랜덤 포레스트 및 뛰어난 부스팅 알고리즘들은 모두 결정 트리 알고리즘을 기반 알고리즘으로 적용함 

- 결정 트리의 단점인 과적합(오버 피팅)을 수십 ~ 수천개의 많은 분류기를 결합해 보완하고 장점인 직관적인 분류 기준은 강화됨 



---

### 보팅(Voting)과 배깅(Bagging) 개요 

- 보팅 배깅은 여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식 

- 보팅과 배깅의 다른점은 보팅의 경우 일반적으로 서로 다른 알고리즘을 가진 분류기를 결합하는 것 .

- 배깅: 각각의 분류기가 모두 같은 유형의 알고리즘 기반이지만 데이터 샘플링을 서로 다르게 가져가면서 학습을 수행해 보팅을 수행하는 것 



---

### 보팅 유형 - 하드보팅, 소프트 보팅 

- Hard Voting : 다수의 Classifier 간 다수결로 최종 class 결정 


      Class 값 1로 예측 
        Classifier 1,3,4  = 1 
        Classifier 2 = 2 
        출력 1 

- Soft Volting : 다수의 Classifier 들의 class 확률을 평균하여 결정 

      Class 값 1로 예측 

       클래스 값 1일 확률 : 0.65 
       클래스 값 2일 확률 : 0.35 

       출력 1 

      predict_proba() 메소드를 이용하여 class 별 확률 결정 

In [1]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings 

warnings.filterwarnings("ignore")

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data , columns=cancer.feature_names)

data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [2]:
# 로지스텍 회귀와 KNN 은 개별 모델 

lr_clf = LogisticRegression(solver="liblinear")
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기 

vo_clf = VotingClassifier( estimators=[('LR',lr_clf),('KNN',knn_clf)] , voting='soft' )

X_train, X_test , y_train , y_test = train_test_split(cancer.data , cancer.target,
                                                      test_size = 0.2 , random_state = 156)

# VotingClassifier 학습/예측/평가 

vo_clf.fit(X_train, y_train)

pred= vo_clf.predict(X_test)
print("Voting 분류기 정확도 : {0:.4f}".format(accuracy_score(y_test , pred )))

# 개별 모델의 학습 / 예측 / 평가 
classifiers = [lr_clf, knn_clf ]

for classifier in classifiers:
  classifier.fit(X_train, y_train)
  pred = classifier.predict(X_test)
  class_name = classifier.__class__.__name__
  print('{0} 정확도 : {1:.4f}' . format(class_name, accuracy_score(y_test, pred )))

Voting 분류기 정확도 : 0.9561
LogisticRegression 정확도 : 0.9474
KNeighborsClassifier 정확도 : 0.9386




---
### 배깅(Bagging) - 랜덤 포레스트 

- 배깅의 대표적인 알고리즘은 랜덤 포레스트

- 앙상블 알고리즘중 비교적 빠른 수행 속도를 가지고 있으며, 다양한 영역에서 높은 예측 성능을 보임 

- 여러개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링하여 개별적으로 학습을 수행한 뒤 최종적으로 모든 분류기가 보팅을 통하여 예측 결정을 하게됨 ㅡ



---

### 랜덤 포레스트의 부트스트래핑 분할 

- 랜덤 프레스트는 개별적인 분류기의 기반 알고리즘은 결정 트리이지만 개별 트리가 학습하는 데이터 세트는 전체 데이터에서 일부가 중첩되게 샘플링된 데이터. 이렇게 여러 개의 세트를 중첩되게 분리하는 것을 부트스트래핑(Bootstrapping)분할 방식이라 함 . ( 그래서 Bagging이 Bootstrap aggregating의 줄임말) 

- 원본 데이터의 건수가 10개인 학습 데이터 세트에 랜덤 포레스트를 3개 결정 트리 기반으로 학습하려고  n_estimators=3으로 하이퍼 파라미터를 부여하면 다음과 같이 데이터 서브 세트가 만들어짐 


                       1 2 3 4 5 6 7 8 9 10  

                      =>  1 2 3 3 3 5 6 8 8 9

                      =>  1 3 4 5 6 8 8 9 9 10

                      =>  1 1 3 4 4 5 6 6 9 9

### 사이킷런 랜덤 포레스트 하이퍼 파라미터 

- n_estimators : 결정 트리의 개수를 지정, 디폴트 값은 100개 , 증가시킬수록 성능이 무조건 향상되는것은 아니며, 늘릴수록 시간 오래걸림 

- max_features 결정 트리에 사용된 max_features와 같음, 하지만 RandomForestClassifier의 기본 max_features는 None이 아니라 auto 즉 sqrt 와 같음, 따라서 랜덤 포레스트의 트리를 분할하는 피처를 참조할 때 전체 피처가 아니라 sqrt(전체 피처 개수)만큼 참조함 (전체 피처가 16개라면 분할을 위해 4개 참조)

- max_depth나 min_samples_leaf와 같이 결정 트리에서 과적합을 개선하기 위해 사용되는 파라미터가 랜덤 포레스트에도 똑같이 적용될 수 있음 

